# Deep Learning Model Training (DNN)
## Stage 06: PyTorch Neural Network for Cryptocurrency Price Prediction

This notebook explores training a Deep Neural Network (DNN) using PyTorch for predicting cryptocurrency prices.

In [ ]:
import os
os.chdir('../')
%pwd

## 1. Configuration Entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DeepModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    hidden_layers: list
    dropout_rate: float
    learning_rate: float
    batch_size: int
    epochs: int
    early_stopping_patience: int
    target_column: str

## 2. Configuration Manager

In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_deep_model_trainer_config(self) -> DeepModelTrainerConfig:
        config = self.config.deep_model_trainer
        params = self.params.DeepModel
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        deep_model_trainer_config = DeepModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            hidden_layers=params.hidden_layers,
            dropout_rate=params.dropout_rate,
            learning_rate=params.learning_rate,
            batch_size=params.batch_size,
            epochs=params.epochs,
            early_stopping_patience=params.early_stopping_patience,
            target_column=schema.name
        )

        return deep_model_trainer_config

## 3. Deep Neural Network Architecture

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import joblib
import json
from mlProject import logger

In [ ]:
class CryptoPriceNet(nn.Module):
    """Neural Network for Cryptocurrency Price Prediction"""
    
    def __init__(self, input_size, hidden_layers, dropout_rate=0.2):
        super(CryptoPriceNet, self).__init__()
        
        layers_list = []
        prev_size = input_size
        
        # Hidden layers
        for hidden_size in hidden_layers:
            layers_list.append(nn.Linear(prev_size, hidden_size))
            layers_list.append(nn.ReLU())
            layers_list.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size
        
        # Output layer
        layers_list.append(nn.Linear(prev_size, 1))
        
        self.network = nn.Sequential(*layers_list)
    
    def forward(self, x):
        return self.network(x)

In [ ]:
class CryptoDataset(Dataset):
    """Custom Dataset for Cryptocurrency Data"""
    
    def __init__(self, features, targets):
        self.features = torch.FloatTensor(features)
        self.targets = torch.FloatTensor(targets).reshape(-1, 1)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

## 4. Deep Model Trainer Component

In [ ]:
class DeepModelTrainer:
    def __init__(self, config: DeepModelTrainerConfig):
        self.config = config
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")
        
    def train(self):
        """Train Deep Neural Network model"""
        try:
            # Load data
            logger.info("Loading training and test data...")
            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)
            
            # Prepare features and target
            train_x = train_data.drop([self.config.target_column], axis=1)
            test_x = test_data.drop([self.config.target_column], axis=1)
            train_y = train_data[self.config.target_column].values
            test_y = test_data[self.config.target_column].values
            
            input_size = train_x.shape[1]
            logger.info(f"Input features: {input_size}")
            logger.info(f"Training samples: {len(train_x)}, Test samples: {len(test_x)}")
            
            # Scale features
            logger.info("Scaling features...")
            scaler = StandardScaler()
            train_x_scaled = scaler.fit_transform(train_x)
            test_x_scaled = scaler.transform(test_x)
            
            # Save scaler
            scaler_path = os.path.join(self.config.root_dir, 'scaler.joblib')
            joblib.dump(scaler, scaler_path)
            logger.info(f"Scaler saved to {scaler_path}")
            
            # Create datasets and dataloaders
            train_dataset = CryptoDataset(train_x_scaled, train_y)
            test_dataset = CryptoDataset(test_x_scaled, test_y)
            
            train_loader = DataLoader(
                train_dataset, 
                batch_size=self.config.batch_size,
                shuffle=True
            )
            test_loader = DataLoader(
                test_dataset,
                batch_size=self.config.batch_size,
                shuffle=False
            )
            
            # Initialize model
            model = CryptoPriceNet(
                input_size=input_size,
                hidden_layers=self.config.hidden_layers,
                dropout_rate=self.config.dropout_rate
            ).to(self.device)
            
            logger.info(f"Model Architecture:\n{model}")
            
            # Loss and optimizer
            criterion = nn.MSELoss()
            optimizer = optim.Adam(
                model.parameters(), 
                lr=self.config.learning_rate,
                weight_decay=1e-5
            )
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=5
            )
            
            # Training loop
            best_loss = float('inf')
            patience_counter = 0
            train_losses = []
            test_losses = []
            
            logger.info("Starting training...")
            for epoch in range(self.config.epochs):
                # Training phase
                model.train()
                train_loss = 0.0
                
                for batch_features, batch_targets in train_loader:
                    batch_features = batch_features.to(self.device)
                    batch_targets = batch_targets.to(self.device)
                    
                    # Forward pass
                    optimizer.zero_grad()
                    outputs = model(batch_features)
                    loss = criterion(outputs, batch_targets)
                    
                    # Backward pass
                    loss.backward()
                    optimizer.step()
                    
                    train_loss += loss.item()
                
                train_loss /= len(train_loader)
                train_losses.append(train_loss)
                
                # Validation phase
                model.eval()
                test_loss = 0.0
                
                with torch.no_grad():
                    for batch_features, batch_targets in test_loader:
                        batch_features = batch_features.to(self.device)
                        batch_targets = batch_targets.to(self.device)
                        
                        outputs = model(batch_features)
                        loss = criterion(outputs, batch_targets)
                        test_loss += loss.item()
                
                test_loss /= len(test_loader)
                test_losses.append(test_loss)
                
                # Learning rate scheduling
                scheduler.step(test_loss)
                
                # Log progress
                if (epoch + 1) % 10 == 0:
                    logger.info(
                        f"Epoch [{epoch+1}/{self.config.epochs}] "
                        f"Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}"
                    )
                
                # Early stopping
                if test_loss < best_loss:
                    best_loss = test_loss
                    patience_counter = 0
                    
                    # Save best model
                    model_path = os.path.join(self.config.root_dir, self.config.model_name)
                    torch.save(model.state_dict(), model_path)
                    logger.info(f"Best model saved with test loss: {best_loss:.6f}")
                else:
                    patience_counter += 1
                    
                if patience_counter >= self.config.early_stopping_patience:
                    logger.info(f"Early stopping at epoch {epoch+1}")
                    break
            
            # Save model configuration
            model_config = {
                'input_size': input_size,
                'hidden_layers': self.config.hidden_layers,
                'dropout_rate': self.config.dropout_rate,
                'feature_names': list(train_x.columns)
            }
            
            config_path = os.path.join(self.config.root_dir, 'model_config.json')
            with open(config_path, 'w') as f:
                json.dump(model_config, f, indent=4)
            logger.info(f"Model config saved to {config_path}")
            
            # Save training history
            history = {
                'train_losses': train_losses,
                'test_losses': test_losses,
                'best_loss': best_loss,
                'epochs_trained': len(train_losses)
            }
            
            history_path = os.path.join(self.config.root_dir, 'training_history.json')
            with open(history_path, 'w') as f:
                json.dump(history, f, indent=4)
            logger.info(f"Training history saved to {history_path}")
            
            logger.info("Training completed successfully!")
            logger.info(f"Best test loss: {best_loss:.6f}")
            
        except Exception as e:
            logger.exception(f"Error during deep model training: {str(e)}")
            raise e

## 5. Execute Training Pipeline

In [ ]:
try:
    config = ConfigurationManager()
    deep_model_trainer_config = config.get_deep_model_trainer_config()
    deep_model_trainer = DeepModelTrainer(config=deep_model_trainer_config)
    deep_model_trainer.train()
except Exception as e:
    raise e

## 6. Visualize Training History

In [ ]:
import matplotlib.pyplot as plt

# Load training history
with open('artifacts/deep_model_trainer/training_history.json', 'r') as f:
    history = json.load(f)

# Plot losses
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history['train_losses'], label='Train Loss')
plt.plot(history['test_losses'], label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Test Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history['test_losses'], label='Test Loss', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Test Loss Over Time')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Best Loss: {history['best_loss']:.6f}")
print(f"Epochs Trained: {history['epochs_trained']}")